In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)

In [ ]:
husk_res = Path("results") / "husky"
vulp_res = Path("results") / "vulpi"

# hbox : metrics/husky/CNN-1700-nohamming.dat
# hham : metrics/husky/CNN-1700-hamming.dat
# vbox : metrics/vulpi/CNN-1700-optim6.dat
# vham : metrics/vulpi/CNN-1700-hamming.dat

paths = {
    "husky": {
        "boxcar": husk_res / "results_CNN_nohamming_mw_1.7.npy",
        "hamming": husk_res / "results_CNN_hamming_mw_1.7.npy",
    },
    "vulpi": {
        "boxcar": vulp_res / "results_CNN_optim6_mw_1.7.npy",
        "hamming": vulp_res / "results_CNN_hamming_mw_1.7.npy",
    },
}

values = {}
values["husky"] = {
    "boxc": np.load(paths["husky"]["boxcar"], allow_pickle=True).item(),
    "hamm": np.load(paths["husky"]["hamming"], allow_pickle=True).item(),
}
values["vulpi"] = {
    "boxc": np.load(paths["vulpi"]["boxcar"], allow_pickle=True).item(),
    "hamm": np.load(paths["vulpi"]["hamming"], allow_pickle=True).item(),
}

In [ ]:
def convert_to_str(x, terrains):
    if x.dtype.type is np.int64:
        return terrains[x]
    return x


def compute_metrics(clf_values):
    terrains = np.array([t for t in clf_values["terrains"] if t != "MIXED"])
    inv_terrains = {t: i for i, t in enumerate(terrains)}
    ypred = convert_to_str(clf_values["pred"], terrains)
    ytest = convert_to_str(clf_values["true"], terrains)
    metrics = pd.Series(
        {
            "accuracy": accuracy_score(
                ytest,
                ypred,
            ),
            "precision": precision_score(
                ytest,
                ypred,
                labels=terrains,
                average=None,
            ),
            "recall": recall_score(
                ytest,
                ypred,
                labels=terrains,
                average=None,
            ),
            "f1-score": f1_score(
                ytest,
                ypred,
                labels=terrains,
                average=None,
            ),
        }
    )
    return metrics

In [ ]:
df = pd.DataFrame(
    {
        "vbox": compute_metrics(values["vulpi"]["boxc"]),
        "vham": compute_metrics(values["vulpi"]["hamm"]),
        "hbox": compute_metrics(values["husky"]["boxc"]),
        "hham": compute_metrics(values["husky"]["hamm"]),
    }
)
df

In [ ]:
vulp_diff = df.vham - df.vbox
vulp_diff.accuracy, vulp_diff.precision

In [ ]:
hamm_diff = df.hham - df.hbox
hamm_diff.accuracy, hamm_diff.precision